# Integration and reference mapping with multigrate

In this notebook, we demonstrate how to use Multigrate with scArches: we build a trimodal reference atlas with Multigrate by integrating CITE-seq and multiome data, and map unimodal as well as multimodal queries onto the reference. We use publically available datasets from NeurIPS 2021 workshop https://openproblems.bio/neurips_2021/.

In [1]:
import scarches as sca
import scanpy as sc
import anndata as ad
import numpy as np
import muon
import gdown
import json
import argparse
import warnings
import pandas as pd
import os
import multigrate as mtg
warnings.filterwarnings("ignore")

sc.set_figure_params(figsize=(4, 4), fontsize=8)
os.environ['CUDA_VISIBLE_DEVICES']='3'

 captum (see https://github.com/pytorch/captum).
INFO:lightning_fabric.utilities.seed:Global seed set to 0
/root/anaconda3/envs/multigrate/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/anaconda3/envs/multigrate/lib/python3.10/site-packages/flax/core/frozen_dict.py:169: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(


In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--task', type=str, default='wnn_cellmask',
    help="Choose a task")
parser.add_argument('--experiment', type=str, default='ref_mapping',
    help="")
parser.add_argument('--kl', type=float, default=1e-1,
    help="")
parser.add_argument('--integ', type=int, default=3000,
    help="")
parser.add_argument('--inner', type=int, default=1,
    help="")
# o = parser.parse_args()
o, _ = parser.parse_known_args() 

task = o.task
data_root = '/dev/shm/processed/'+task
print(o.task)

data = {'rna':[], 'adt':[], 'atac':[]}
layers = {'rna':[], 'adt':[], 'atac':[]}
# labels = []
mask = {'rna':[], 'adt':[]}
mods = []

def creat_h5ad(mat, obs_names, label, batch):
    adata = sc.AnnData(mat)
    adata.obs_names = obs_names
    # adata.obs['l1'] = label
    adata.obs['batch'] = batch
    adata.obs['batch']  = adata.obs['batch'].astype('category')
    return adata

for i in range(8):
    m = []
    d = {}
    p = data_root+'/subset_'+str(i)
    
    # label = pd.read_csv(label_path[i], sep=',', index_col=0)
    for m in ['rna', 'adt', 'atac']:
        if os.path.exists(p+'/mat/'+m+'.csv'):
            mods.append(m)
            print(i, m)

            mat = pd.read_csv(p+'/mat/'+m+'.csv', sep=',', index_col=0)
            cellnames = pd.read_csv(p+'/cell_names.csv', sep=',', index_col=0).values.flatten()
            # label.index = cellnames
            adata = creat_h5ad(mat, cellnames, None, i)

            print(adata)

            if m != 'atac':
                mask[m].append(pd.read_csv(p+'/mask/'+m+'.csv', index_col=0))
            if m=='adt':
                adata.layers['counts'] = adata.X.copy()
            elif m=='rna':
                adata.layers['counts'] = adata.X.copy()
            elif m=='atac':
                adata.layers['counts'] = adata.X.copy()
                
            data[m].append(adata)
        else:
            data[m].append(None)
        

# %%
if o.inner:
    final_mask = {'rna':None, 'adt':None}
    for m in ['rna', 'adt']:
        for mk in  mask[m]:
            if final_mask[m] is None:
                final_mask[m] = mk.values.astype('bool')[0]
            else:
                final_mask[m] = final_mask[m] & mk.values.astype('bool')[0]
        print(final_mask[m].sum())

    rna_dim = None
    for m in ['rna', 'adt']:
        for i in range(len(data[m])):
            if data[m][i] is not None:
                data[m][i] = data[m][i][:, final_mask[m]]   
                print(data[m][i])

for m in ['rna', 'adt', 'atac']:
    for i in range(8):
        if data[m][i] is not None and m=='rna':
            rna = data[m][i]
            sc.pp.normalize_total(rna, target_sum=1e4)
            sc.pp.log1p(rna)
            layers[m].append('counts')

        elif data[m][i] is not None and m=='adt':
            adt = data[m][i]
            adt.X = adt.layers['counts'].copy()
            muon.prot.pp.clr(adt)
            print(np.max(adt.X))
            adt.layers['clr'] = adt.X.copy()
            layers[m].append('clr')

        elif data[m][i] is not None and m=='atac':
            atac = data[m][i]
            muon.atac.pp.tfidf(atac, scale_factor=1e4)
            atac
            atac.layers['tf-idf'] = atac.X.copy()
            atac.X = atac.layers['counts'].copy()
            sc.pp.normalize_total(atac, target_sum=1e4)
            sc.pp.log1p(atac)
            atac.layers['log-norm'] = atac.X.copy()
            layers[m].append('log-norm')

        else:
            layers[m].append(None)
print(data)
print(layers)

# %%
mod = []
losses = []
for i in ['rna', 'atac', 'adt']:
    if i in mods:
        mod.append(i)
        if i=='rna':
            losses.append('nb')
        else:
            losses.append('mse')

# adata = sca.models.organize_multiome_anndatas(
#     adatas = [data['rna'], data['atac'], data['adt']],    # a list of anndata objects per modality, RNA-seq always goes first
#     layers = [layers['rna'], layers['atac'], layers['adt']], # if need to use data from .layers, if None use .X
# )


wnn_cellmask
0 rna


AnnData object with n_obs × n_vars = 6142 × 4050
    obs: 'batch'
0 adt
AnnData object with n_obs × n_vars = 6142 × 224
    obs: 'batch'
1 rna
AnnData object with n_obs × n_vars = 3978 × 4050
    obs: 'batch'
1 adt
AnnData object with n_obs × n_vars = 3978 × 224
    obs: 'batch'
2 rna
AnnData object with n_obs × n_vars = 4103 × 4050
    obs: 'batch'
2 adt
AnnData object with n_obs × n_vars = 4103 × 224
    obs: 'batch'
3 rna
AnnData object with n_obs × n_vars = 5172 × 4050
    obs: 'batch'
3 adt
AnnData object with n_obs × n_vars = 5172 × 224
    obs: 'batch'
4 rna
AnnData object with n_obs × n_vars = 4205 × 4050
    obs: 'batch'
4 adt
AnnData object with n_obs × n_vars = 4205 × 224
    obs: 'batch'
5 rna
AnnData object with n_obs × n_vars = 5265 × 4050
    obs: 'batch'
5 adt
AnnData object with n_obs × n_vars = 5265 × 224
    obs: 'batch'
6 rna
AnnData object with n_obs × n_vars = 8770 × 4050
    obs: 'batch'
6 adt
AnnData object with n_obs × n_vars = 8770 × 224
    obs: 'batch'
7 rna

In [55]:
# label_path = [
#     './data/sct/atac+rna+adt/dogma_label_mask/P1/label_seurat/l1.csv',
#     './data/sct/atac+rna+adt/dogma_label_mask/P2/label_seurat/l1.csv',
#     './data/sct/atac+rna+adt/dogma_label_mask/P3/label_seurat/l1.csv',
#     './data/sct/atac+rna+adt/dogma_label_mask/P4/label_seurat/l1.csv',
#     './data/sct/atac+rna+adt/tea_label_mask/P5/label_seurat/l1.csv',
#     './data/sct/atac+rna+adt/tea_label_mask/P6/label_seurat/l1.csv',
#     './data/sct/atac+rna+adt/tea_label_mask/P7/label_seurat/l1.csv',
#     './data/sct/atac+rna+adt/tea_label_mask/P8/label_seurat/l1.csv']
# labels = []
# for i in label_path:
#     labels.append(pd.read_csv(i, index_col=0))

In [4]:
cell_names_all = []
for i in range(8):
    p = data_root+'/subset_'+str(i)
    cell_names_all.append(pd.read_csv(p+'/cell_names.csv', sep=',', index_col=0).values.flatten())

In [56]:
# for i in range(8):
#     labels[i].index = cell_names_all[i]

## Initialize the model

Next, we initialize the model. If using raw counts for RNA-seq, use NB loss, if normalized counts, use MSE. For ADT we use CLR-normalized counts and MSE loss. We need to specify `mmd='marginal'` and set the coeficient to the integration loss if we want to later map unimodal data onto this reference.

## Map and Query


## create triple modality model

In [3]:
# rna = data['rna']
# adt = data['adt']
# atac = data['atac']

In [6]:
adata_all = mtg.data.organize_multiome_anndatas(
    adatas = [data['rna'],  data['adt']],    # a list of anndata objects per modality, RNA-seq always goes first
    layers = [layers['rna'], layers['adt']], # if need to use data from .layers, if None use .X
)
print(adata_all)

AnnData object with n_obs × n_vars = 46213 × 3675
    obs: 'batch', 'group'
    var: 'modality'
    uns: 'modality_lengths'
    layers: 'counts'


In [7]:
adata_all = adata_all[np.concatenate(cell_names_all)]

In [64]:
adata_all.obs['label'] = pd.concat(labels)

In [8]:
adata_ref = adata_all[(adata_all.obs['batch']==0)]
print(adata_ref)

View of AnnData object with n_obs × n_vars = 6142 × 3675
    obs: 'batch', 'group'
    var: 'modality'
    uns: 'modality_lengths'
    layers: 'counts'


In [9]:
sca.models.MultiVAE.setup_anndata(
    adata_ref,
    categorical_covariate_keys=['batch'],
    rna_indices_end=adata_ref.uns['modality_lengths'][0],
)

In [11]:
model = sca.models.MultiVAE(
    adata_ref,
    losses=['nb', 'mse',],
    loss_coefs={'kl': 1e-1,
                'integ': 3000,
                },
    z_dim=32,
    integrate_on='batch',
    mmd='marginal')

In [12]:
model.train(use_gpu=True, max_epochs=500)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]


Epoch 500/500: 100%|██████████| 500/500 [06:47<00:00,  1.24it/s, loss=1.35e+03, v_num=1]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=500` reached.


Epoch 500/500: 100%|██████████| 500/500 [06:47<00:00,  1.23it/s, loss=1.35e+03, v_num=1]


In [14]:
model.get_latent_representation()
os.makedirs('./result/wnn_cellmask/multigrate_obo/default/predict/subset_0/z/joint/')
pd.DataFrame(adata_ref[adata_ref.obs['batch']==0].obsm['latent']).to_csv('./result/wnn_cellmask/multigrate_obo/default/predict/subset_0/z/joint/00.csv', index=False, header=False)

In [186]:
# model2.save('./result/teadog_paired_full_label_mask_new/multigrate_obo/train')

In [16]:
model2 = model
for i in range(1, 8):
    adata_q = adata_all[(adata_all.obs['batch']==i)]
    sca.models.MultiVAE.setup_anndata(
    adata_q,
    categorical_covariate_keys=['batch'],
    rna_indices_end=adata_q.uns['modality_lengths'][0])
    q_model = sca.models.MultiVAE.load_query_data(adata_q, model2)
    q_model.train(weight_decay=0)
    q_model.get_latent_representation(adata=adata_q)
    if not os.path.exists('./result/wnn_cellmask/multigrate_obo/default/predict/subset_%d/z/joint/'%(i)):
        os.makedirs('./result/wnn_cellmask/multigrate_obo/default/predict/subset_%d/z/joint/'%(i))
    pd.DataFrame(adata_q.obsm['latent']).to_csv('./result/wnn_cellmask/multigrate_obo/default/predict/subset_%d/z/joint/00.csv'%(i), index=False, header=False)
    model2 = q_model

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]


Epoch 200/200: 100%|██████████| 200/200 [01:08<00:00,  2.83it/s, loss=1.56e+03, v_num=1]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 200/200: 100%|██████████| 200/200 [01:08<00:00,  2.90it/s, loss=1.56e+03, v_num=1]


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]


...

Epoch 200/200: 100%|██████████| 200/200 [02:23<00:00,  1.40it/s, loss=2.34e+03, v_num=1]


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]


Epoch 200/200: 100%|██████████| 200/200 [02:22<00:00,  1.44it/s, loss=2.16e+03, v_num=1]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 200/200: 100%|██████████| 200/200 [02:22<00:00,  1.41it/s, loss=2.16e+03, v_num=1]
